In [1]:
import os
import math
import torch
from dataclasses import dataclass, field

from code.train import train
from code.optimizers import Optimizer
from code.problems import Problem

%matplotlib widget
%load_ext autoreload
%autoreload 1

In [2]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

In [ ]:
n_iters=200, n_peers=11, problem=Logistic, n_samples=4500, h_ratio=0.6, batch_size=35, lr=0.01, md_n_iters=(None, 50), md_lr=(None, 0.05)

In [ ]:
md_n_iters=(None, 50), md_lr=(None, 0.05)

In [89]:
@dataclass
class Config():
    n_iters: int = 200
    n_peers: int = 11
    seed: int = 0

    # game: Game = Game.Quadratic
    problem: Problem = field(default_factory=lambda: Problem.Logistic)
    n_samples: int = 4500
    h_ratio: float = 0.6
    
    optimizer: Optimizer = field(default_factory=lambda: Optimizer.SGD)
    batch_size: int = 35
    lr: float = 1e-2
    
    true_weights: bool = None
    
    md_n_iters_: int = None
    md_full_: bool = None
    md_lr_: int = None

In [90]:
config = Config()
config.optimizer = Optimizer.SGD
config.true_weights = False
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
train(config)

Trying port 9920
2
skdjf 2962
rank 0: n_samples 4500
rank 1: n_samples 4500       
rank 2: n_samples 4500
rank 4: n_samples 4500
rank 3: n_samples 4500
rank 5: n_samples 4500
rank 6: n_samples 4500
rank 7: n_samples 4500
rank 8: n_samples 4500
rank 9: n_samples 4500
2
skdjf 2960
rank 10: n_samples 4500


In [ ]:
config = Config()
config.optimizer = Optimizer.SGD
config.true_weights = True
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
train(config)

Trying port 23760
2
skdjf 2962
rank 0: n_samples 4500


In [ ]:
config = Config()
config.optimizer = Optimizer.SGDMD
config.md_full_ = False
config.md_n_iters_ = 50
config.md_lr_ = 0.05
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
train(config)

In [ ]:
config = Config()
config.optimizer = Optimizer.SGDMD
config.md_full_ = True
config.md_n_iters_ = 50
config.md_lr_ = 0.05
os.environ['MLFLOW_RUN_NAME'] = config.optimizer.name
train(config)